In [ ]:
import cv2
import numpy as np

# Region detector

In [ ]:
import cv2
import numpy as np

def detect_mouth(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)

    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    if len(faces) == 0:
        print("No faces detected.")
        return image, None
    
    (x, y, w, h) = faces[0]
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
    mouth_roi_y_start = int(y + 0.6 * h)
    mouth_roi = gray[mouth_roi_y_start: y + h, x: x + w]
  
    mouth_cascade = cv2.CascadeClassifier('haarcascade_mcs_mouth.xml')
    mouths = mouth_cascade.detectMultiScale(mouth_roi, scaleFactor=1.1, minNeighbors=5)
    
    if len(mouths) == 0:
        print("No mouth detected.")
        return image, None

    best_candidate = None
    best_y = -1
    for (mx, my, mw, mh) in mouths:
        absolute_y = mouth_roi_y_start + my 
        if absolute_y > best_y:
            best_y = absolute_y
            best_candidate = (mx, my, mw, mh)

    if best_candidate is None:
        print("No mouth detected.")
        return image, None

    mx, my, mw, mh = best_candidate
    mouth_x = x + mx
    mouth_y = mouth_roi_y_start + my
    cv2.rectangle(image, (mouth_x, mouth_y), (mouth_x + mw, mouth_y + mh), (0, 255, 0), 2)
    detected_mouth = (mouth_x, mouth_y, mw, mh)

    return image, detected_mouth

video_capture = cv2.VideoCapture('test.mp4')
if not video_capture.isOpened():
    print("Error: Could not open video file.")
    exit()

frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video_capture.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
output_video = cv2.VideoWriter('detected_mouth_output.mp4', fourcc, fps, (frame_width, frame_height))

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    output_frame, mouth_coords = detect_mouth(frame)
    output_video.write(output_frame)

# Clean up
video_capture.release()
output_video.release()